In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras import backend as K


In [14]:
# Focal Loss function
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        pt = K.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
        return -K.sum(alpha * K.pow(1.0 - pt, gamma) * K.log(pt))

    return focal_loss_fixed

In [10]:
# Load dataset
file_path = 'D:\Tugas Akhir\Stroke\data_oversampled.csv'
df = pd.read_csv(file_path)

# Pisahkan fitur dan target
X = df.drop('stroke', axis=1)
y = df['stroke']

In [11]:
df

,gender,age,hypertension,heart_disease,ever_married,work_type_Govt_job,work_type_Private,work_type_Self-employed,Residence_type,avg_glucose_level,bmi,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes,stroke
0,0,"0,7368",0,1,1,0,1,0,0,"0,8012","0,5227",0,1,0,0,1
1,0,"0,9649",0,1,1,0,1,0,1,"0,2342","0,438",0,0,1,0,1
2,1,"0,4211",0,0,1,0,1,0,0,"0,5358","0,4773",0,0,0,1,1
3,1,"0,9474",1,0,1,0,0,1,1,"0,5491","0,2624",0,0,1,0,1
4,0,"0,9825",0,0,1,0,1,0,0,"0,605","0,3657",0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5766,1,"0,8804",0,0,1,0,1,0,1,"0,6311","0,3073",1,0,0,0,1
5767,0,"0,8583",0,0,1,0,1,0,1,"0,1403","0,2548",1,0,0,0,1
5768,1,"0,9298",0,0,1,0,1,0,0,"0,0904","0,1904",1,0,0,0,1
5769,1,"0,9009",0,0,1,0,1,0,1,"0,2097","0,2386",1,0,0,0,1


In [12]:
# Hitung frekuensi nilai unik dalam kolom 'stroke'
stroke_counts = df['stroke'].value_counts()
# Tampilkan output
print("Frekuensi nilai unik dalam kolom 'stroke':")
print(stroke_counts)

Frekuensi nilai unik dalam kolom 'stroke':
stroke
0    3481
1    2290
Name: count, dtype: int64


In [15]:
# Pisahkan data menjadi set pelatihan dan pengujian
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
X_train

,gender,age,hypertension,heart_disease,ever_married,work_type_Govt_job,work_type_Private,work_type_Self-employed,Residence_type,avg_glucose_level,bmi,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
4371,0,"0,5087",0,0,1,0,1,0,1,"0,1929","0,3679",0,0,0,1
2380,1,"0,3509",0,0,1,0,0,1,1,"0,0672","0,2893",0,0,0,1
2337,1,"0,8421",0,0,1,1,0,0,1,"0,0495","0,3926",0,0,1,0
2437,0,"0,2105",0,0,1,1,0,0,0,"0,1241","0,5041",0,0,0,1
3903,1,"0,7303",0,0,1,0,1,0,1,"0,3203","0,3731",0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,1,"0,9747",0,0,1,0,1,0,1,"0,1293","0,3551",0,0,1,0
5191,0,"0,9217",0,1,1,0,1,0,0,"0,3821","0,3603",0,0,0,1
5226,0,"0,8056",0,0,1,0,1,0,1,"0,3716","0,4218",0,1,0,0
5390,0,"0,8885",0,0,0,0,0,1,0,"0,1201","0,3187",0,0,1,0


In [18]:
# Hitung frekuensi nilai unik dalam kolom 'stroke' pada data pelatihan
stroke_counts_train = y_train.value_counts()
# Tampilkan output
print("Frekuensi nilai unik dalam kolom 'stroke' pada data pelatihan:")
print(stroke_counts_train)

Frekuensi nilai unik dalam kolom 'stroke' pada data pelatihan:
stroke
0    2788
1    1828
Name: count, dtype: int64
